In [1]:
import sys
sys.path.append('../') 

In [2]:
%load_ext autoreload
%autoreload 2
import sklearn
import copy
import numpy as np

import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns 
# from viz import viz
from bokeh.plotting import figure, show, output_notebook, output_file, save
from functions import merge_data
from sklearn.model_selection import RandomizedSearchCV
import load_data


from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from fit_and_predict import fit_and_predict

In [3]:
preds_df = pd.read_pickle("multi_day_2.pkl")

In [4]:
def l1(arr1,arr2):
    return sum([np.abs(a1-a2) for (a1,a2) in zip(arr1,arr2)])/len(arr1)

In [5]:
outcome = np.array([preds_df['deaths'].values[i][-1] for i in range(len(preds_df))])
residuals = {}
for days_ahead in [1, 2, 3]:
    for lower_threshold in [0, 10]:
        colname = f'{days_ahead} day, deaths>={lower_threshold}'
        residuals[colname] = []
        for method in ['exponential', 'shared_exponential', 'ensemble']:
            key = f'predicted_deaths_{method}_{days_ahead}'
            preds = [p[-1] for p in preds_df[key][outcome > lower_threshold]]
            residuals[colname].append(l1(outcome[outcome > lower_threshold],preds))
            if method == 'shared_exponential':
                key = f'predicted_deaths_{method}_{days_ahead}_demographics'
                preds = [p[-1] for p in preds_df[key][outcome > lower_threshold]]
                residuals[colname].append(l1(outcome[outcome > lower_threshold],preds))    

In [6]:
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_max]

In [7]:
res_df = pd.DataFrame(residuals, index=['exponential', 'shared', 'demographics', 'ensemble'])
res_df = res_df.astype(float).round(2)
res_df.style.highlight_min().format("{:.2f}")

,"1 day, deaths>=0","1 day, deaths>=10","2 day, deaths>=0","2 day, deaths>=10","3 day, deaths>=0","3 day, deaths>=10"
exponential,1.34,9.67,2.29,14.84,3.63,22.91
shared,1.80,6.49,3.83,11.70,6.08,12.35
demographics,2.09,7.85,4.57,7.87,7.88,10.84
ensemble,1.46,7.89,2.44,12.50,3.21,10.15


In [8]:
import plotly.express as px

In [9]:
outcome = np.array([preds_df['deaths'].values[i][-1] for i in range(len(preds_df))])
preds_df['true_outcome'] = outcome
print(np.array([p[-1] for p in preds_df['predicted_deaths_ensemble_3']]))
preds_df['3_day_ahead_pred'] = [p[-1] for p in preds_df['predicted_deaths_ensemble_3']]
preds_df = preds_df[preds_df.true_outcome > 10]
fig = px.scatter(preds_df, x='true_outcome', y='3_day_ahead_pred')

[280.47081928 164.04023725 175.94218632 ...   2.14455634   2.14455634
   2.14455634]


In [10]:
preds_df.keys()

Index(['predicted_deaths_ensemble_1', 'predicted_deaths_ensemble_3',
       'predicted_deaths_exponential_1',
       'predicted_deaths_shared_exponential_1',
       'predicted_deaths_shared_exponential_1_demographics',
       'predicted_deaths_exponential_2',
       'predicted_deaths_shared_exponential_2',
       'predicted_deaths_shared_exponential_2_demographics',
       'predicted_deaths_ensemble_2', 'predicted_deaths_exponential_3',
       'predicted_deaths_shared_exponential_3',
       'predicted_deaths_shared_exponential_3_demographics', 'countyFIPS',
       'CountyNamew/StateAbbrev', 'deaths', 'true_outcome',
       '3_day_ahead_pred'],
      dtype='object')

In [27]:
preds_df_2 = preds_df[preds_df.true_outcome > 50]
fig = px.scatter(preds_df_2, x='true_outcome', y='3_day_ahead_pred', text='CountyNamew/StateAbbrev')

In [29]:
fig.update_traces(textposition='bottom center')
fig.update_layout(xaxis_type="log", yaxis_type="log")
fig.add_shape(
        # Line reference to the axes
            type="line",
            xref="x",
            yref="y",
            x0=50,
            y0=50,
            x1=320,
            y1=320,
            line=dict(
                color="LightSeaGreen",
                width=3,
            ),
        )

fig.update_layout(
    title="Actual deaths by 3/29 vs. our predictions on 3/26",
    xaxis_title="Actual deaths",
    yaxis_title="3 day ahead prediction",
    font = dict(
        family='sans-serif', 
        size=12,
    )
)

fig.update_layout(
    title={
        'text': "Actual deaths by 3/29 vs. our predictions on 3/26",
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

# change size of names
fig.update_traces(textfont_size=15)


In [30]:
fig.show()

In [31]:
preds_dict = {}
for d in [1, 2, 3, 7]:
    fit_and_predict(train_df, 
                    test_df, 
                    outcome=outcome_type, 
                    mode='eval_mode',
                    method='ensemble',
                    demographic_vars=[],
                    target_day=np.array([d]))
    preds_dict[d] = np.array([p[-1] for p in test_df[f'predicted_deaths_ensemble_{d}']])
preds_dict['FIPS'] = test_df['countyFIPS']
preds_dict['outcome'] = np.array([df[outcome_type].values[i][-1] for i in range(len(df))])
preds_dict['county'] = df['CountyNamew/StateAbbrev'].values
preds_df = pd.DataFrame(preds_dict)
preds_df.to_pickle('multi_day_1.pkl')

NameError: name 'train_df' is not defined